In [60]:
AMOUNT_POLICIES = 2002
print("""
Generating fake data for
- %d policies,
""" % (AMOUNT_POLICIES))


Generating fake data for
- 2002 policies,



In [61]:
from sqlalchemy import create_engine
core_engine = create_engine('mysql+mysqlconnector://sa:sa@localhost:3306/customercore?auth_plugin=caching_sha2_password', echo=False)
core_engine.driver
policy_engine = create_engine('mysql+mysqlconnector://sa:sa@localhost:3306/policymanagement?auth_plugin=caching_sha2_password', echo=False)
policy_engine.driver

'mysqlconnector'

In [62]:
from sqlalchemy import inspect

core_inspection = inspect(core_engine)
print(core_inspection.default_schema_name)
policy_inspection = inspect(policy_engine)
print(policy_inspection.default_schema_name)

customercore
policymanagement


In [63]:
relevant_methods = [x for x in dir(core_inspection) if x.startswith("get")]
relevant_methods

['get_check_constraints',
 'get_columns',
 'get_foreign_keys',
 'get_indexes',
 'get_materialized_view_names',
 'get_multi_check_constraints',
 'get_multi_columns',
 'get_multi_foreign_keys',
 'get_multi_indexes',
 'get_multi_pk_constraint',
 'get_multi_table_comment',
 'get_multi_table_options',
 'get_multi_unique_constraints',
 'get_pk_constraint',
 'get_schema_names',
 'get_sequence_names',
 'get_sorted_table_and_fkc_names',
 'get_table_comment',
 'get_table_names',
 'get_table_options',
 'get_temp_table_names',
 'get_temp_view_names',
 'get_unique_constraints',
 'get_view_definition',
 'get_view_names']

In [64]:
core_inspection.get_table_names()

['addresses',
 'addresses_seq',
 'customer_profile_entity',
 'customer_profile_entity_move_history',
 'customers']

In [65]:
policy_inspection.get_table_names()

['addresses',
 'addresses_seq',
 'customerinfos',
 'customerinfos_seq',
 'insuranceagreementitems',
 'insuranceagreementitems_seq',
 'insuranceoptions',
 'insuranceoptions_seq',
 'insurancepolicies',
 'insurancequoterequests',
 'insurancequoterequests_seq',
 'insurancequoterequests_status_history',
 'insurancequotes',
 'insurancequotes_seq',
 'insuringagreements',
 'insuringagreements_agreement_items',
 'insuringagreements_seq',
 'requeststatuschanges',
 'requeststatuschanges_seq',
 'user_logins',
 'user_logins_seq']

In [66]:
import pandas as pd
customers = pd.read_sql("SELECT * FROM customers", con=core_engine)
print(customers.head())

   customer_profile_id          id
0                    1  q3ccukxtdm
1                    2  qd6r90bgjz
2                    3  yq0itdxwmq
3                    4  6ro92pc7r3
4                    5  t8u3ujoqao


In [67]:
pd.DataFrame(policy_inspection.get_columns('insurancepolicies'))

,name,type,default,comment,nullable,autoincrement
0,deductible_amount,"DECIMAL(38, 2)",None,None,True,NaN
1,deductible_currency,VARCHAR(3),None,None,True,NaN
2,limit_amount,"DECIMAL(38, 2)",None,None,True,NaN
3,limit_currency,VARCHAR(3),None,None,True,NaN
4,premium_amount,"DECIMAL(38, 2)",None,None,True,NaN
5,premium_currency,VARCHAR(3),None,None,True,NaN
6,creation_date,DATETIME,None,None,True,NaN
7,end_date,DATETIME,None,None,True,NaN
8,insuring_agreement_id,BIGINT,None,None,True,False
9,start_date,DATETIME,None,None,True,NaN


In [68]:
from faker import Faker
fake = Faker("en_US")

In [69]:
pd.DataFrame(policy_inspection.get_columns('insuringagreements'))

,name,type,default,comment,nullable,autoincrement
0,id,BIGINT,None,None,False,False


In [70]:
insuringagreements = pd.DataFrame(index=range(2, AMOUNT_POLICIES+2))
insuringagreements.index.name='id'
insuringagreements.head()

""
id
2
3
4
5
6


In [71]:
insuringagreements_seq = pd.DataFrame(index=range(1,2))
insuringagreements_seq['next_val'] = AMOUNT_POLICIES+1
insuringagreements_seq.set_index('next_val', inplace=True)
insuringagreements_seq.head()

""
next_val
2003


In [72]:
pd.DataFrame(policy_inspection.get_columns('insurancepolicies'))

,name,type,default,comment,nullable,autoincrement
0,deductible_amount,"DECIMAL(38, 2)",None,None,True,NaN
1,deductible_currency,VARCHAR(3),None,None,True,NaN
2,limit_amount,"DECIMAL(38, 2)",None,None,True,NaN
3,limit_currency,VARCHAR(3),None,None,True,NaN
4,premium_amount,"DECIMAL(38, 2)",None,None,True,NaN
5,premium_currency,VARCHAR(3),None,None,True,NaN
6,creation_date,DATETIME,None,None,True,NaN
7,end_date,DATETIME,None,None,True,NaN
8,insuring_agreement_id,BIGINT,None,None,True,False
9,start_date,DATETIME,None,None,True,NaN


In [73]:
insurancepolicies = pd.DataFrame(index=range(2, AMOUNT_POLICIES+2))
insurancepolicies.index.name='id'
insurancepolicies['creation_date'] = insurancepolicies.index.map(lambda x : fake.date())
insurancepolicies['customer_id'] = customers.index.repeat(2).to_list()
insurancepolicies['deductible_amount'] = insurancepolicies.index.map(lambda x : fake.numerify("###.##"))
insurancepolicies['deductible_currency'] = insurancepolicies.index.map(lambda x : fake.currency_code())
insurancepolicies['premium_amount'] = insurancepolicies.index.map(lambda x : fake.numerify("###.##"))
insurancepolicies['premium_currency'] = insurancepolicies.index.map(lambda x : fake.currency_code())
insurancepolicies['limit_amount'] = insurancepolicies.index.map(lambda x : fake.numerify("###.##"))
insurancepolicies['limit_currency'] = insurancepolicies.index.map(lambda x : fake.currency_code())
insurancepolicies['end_date'] = insurancepolicies.index.map(lambda x : fake.date())
insurancepolicies['start_date'] = insurancepolicies.index.map(lambda x : fake.date())
insurancepolicies['name'] = "Mock Insurance"
insurancepolicies['insuring_agreement_id'] = insuringagreements.sample(len(insurancepolicies), replace=False).index
insurancepolicies.head()

,creation_date,customer_id,deductible_amount,deductible_currency,premium_amount,premium_currency,limit_amount,limit_currency,end_date,start_date,name,insuring_agreement_id
id,,,,,,,,,,,,
2,2015-03-24,0,295.45,PAB,702.46,BTN,506.80,NGN,2012-12-11,2010-01-29,Mock Insurance,12
3,1974-04-02,0,331.21,XPF,161.26,MOP,634.39,VUV,1991-10-12,2006-08-11,Mock Insurance,1321
4,2024-05-09,1,711.05,MNT,814.86,NAD,098.14,KPW,1996-09-16,2023-02-23,Mock Insurance,312
5,1983-12-10,1,458.83,KWD,271.63,LBP,021.50,GNF,1978-03-21,1994-07-04,Mock Insurance,1960
6,1975-04-14,2,676.28,LRD,900.03,MZN,651.25,IRR,1972-02-01,2008-09-10,Mock Insurance,610


In [74]:
def store(dataframe, table_name):
    dataframe.to_sql(table_name, con=policy_engine, if_exists="append", chunksize=100)

store(insuringagreements, 'insuringagreements')
store(insuringagreements_seq, 'insuringagreements_seq')
store(insurancepolicies, 'insurancepolicies')

print("Done")

Done
